In [1]:
from model import *

device = torch.device('cuda')

context_length = 128
embed_dim = 128
n_head = 4
n_layer = 2

vocab = Vocab()
model = torch.compile(LLM(vocab, context_length, embed_dim, n_head, n_layer).to(device))
model.load_state_dict(torch.load("harry_potter.pt"))

<All keys matched successfully>

# Generate

In [2]:
torch.tensor(vocab.encode("b"), dtype=torch.long)

tensor([65])

In [12]:
g = model.generate(torch.tensor(vocab.encode("Harry"), dtype=torch.long)[None,:].to(device), 100, temperature=1.0, top_k=None)
print(vocab.decode(g[0].tolist()))

Harry paused. “Anyway … we see what’s going on Snape. …”
“We honestly tried to talk to your lines!” said Mr. Malfoy quietly. “I had to come, and we’re never going to —”
“I know it, Mr. Weasley,” said quietly, now packed the field in rain on the untouched people of the dark grounds between their pictures.


 snail on the goldenestrooms, the dementor celebrationilia, played with a locked page in the necklace …
Harry advised him to help them, because a man was among the lamps rolled away from him as they didn’t sound like an idiot. He had deserted living room at his presence, but knew; could I won’t think he’d seen him by a teacher’s coming from the Daily Prophet. but Ron’s fellow students were wearing three years whom he was merely the best they had seen, the shame in the Prime Minister’sSumfive Death Efectll explain why may stare Fred and Professor McGonagall appeared, scanning the Dwaring changes on rain Defense Against theuced with the Death Eaters else in the lamp thatigo leaves were